In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#import wrangle
import env
import seaborn as sns
import acquire
import prep

In [2]:
#single family residence from zillow DB
df = acquire.wrangle_zillow()

In [8]:
#drops columns with < 80% of values filled in
df = prep.drop_columns(df)

In [12]:
df.head()

,transactiondate,id,bathrooms,bedrooms,sqft,tax_value,id,parcelid,bathroomcnt,bedroomcnt,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id,parcelid,logerror,transactiondate
0,2017-01-02,20,2.0,4.0,3633.0,296425.0,20,11324547,2.0,4.0,...,222321.0,296425.0,2016.0,74104.0,6941.39,NaN,20,12325145,0.042463,2017-01-02
1,2017-01-02,31,4.0,3.0,1620.0,847770.0,31,11544747,4.0,3.0,...,339532.0,847770.0,2016.0,508238.0,10244.94,NaN,31,14338242,-0.019546,2017-01-02
2,2017-01-02,33,2.0,3.0,2077.0,646760.0,33,11585547,2.0,3.0,...,210192.0,646760.0,2016.0,436568.0,7924.68,NaN,33,12155445,-0.040807,2017-01-02
3,2017-01-03,110,1.0,3.0,1244.0,169471.0,110,12716947,1.0,3.0,...,108040.0,169471.0,2016.0,61431.0,2532.88,NaN,110,14724525,0.042412,2017-01-03
4,2017-01-03,111,2.0,3.0,1300.0,233266.0,111,12757147,2.0,3.0,...,77415.0,233266.0,2016.0,155851.0,3110.99,NaN,111,14179809,-0.000908,2017-01-03


In [11]:
df.isnull().sum().sort_values(ascending=False)

regionidcity                    1008
lotsizesquarefeet                300
censustractandblock              263
finishedsquarefeet12             186
taxamount                         71
regionidzip                       64
structuretaxvaluedollarcnt        62
yearbuilt                         27
fullbathcnt                       20
calculatedbathnbr                 20
tax_value                          0
sqft                               0
id                                 0
bedrooms                           0
parcelid                           0
bathrooms                          0
bathroomcnt                        0
bedroomcnt                         0
calculatedfinishedsquarefeet       0
id                                 0
fips                               0
transactiondate                    0
latitude                           0
longitude                          0
logerror                           0
propertycountylandusecode          0
propertylandusetypeid              0
r

In [ ]:
df[['landtaxvaluedollarcnt','lotsizesquarefeet']].head()

In [ ]:
df['tax_value_per_foot'] = df.landtaxvaluedollarcnt/df.lotsizesquarefeet
df.tax_value_per_foot.mean()

In [ ]:
df.lotsizesquarefeet.head()

In [ ]:
df.landtaxvaluedollarcnt /df.tax_value_per_foot.mean()

In [ ]:
df.isnull().sum().sort_values(ascending=False)